<a href="https://colab.research.google.com/github/aliciawill/BigData/blob/master/%ED%85%90%EC%84%9C2_%ED%83%80%EC%9D%B4%ED%83%80%EB%8B%89_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import functools

import numpy as np
import tensorflow as tf

In [2]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

24576/13049 [========================================================] - 0s 0us/step


In [3]:
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

In [4]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [5]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [6]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [7]:
show_batch(raw_train_data)

sex                 : [b'male' b'female' b'male' b'male' b'male']
age                 : [25. 35. 17. 28. 24.]
n_siblings_spouses  : [0 1 0 0 2]
parch               : [0 0 0 0 0]
fare                : [ 0.    83.475  7.125  7.896 73.5  ]
class               : [b'Third' b'First' b'Third' b'Third' b'Second']
deck                : [b'unknown' b'C' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'y' b'n' b'y' b'y' b'n']


In [8]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

sex                 : [b'female' b'male' b'female' b'female' b'male']
age                 : [63. 25. 28. 36. 17.]
n_siblings_spouses  : [1 1 0 0 0]
parch               : [0 0 1 0 0]
fare                : [ 77.958   7.775  55.    135.633   8.663]
class               : [b'First' b'Third' b'First' b'First' b'Third']
deck                : [b'D' b'unknown' b'E' b'C' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Cherbourg' b'Southampton']
alone               : [b'n' b'n' b'n' b'y' b'y']


In [9]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 : [20. 30. 23. 28. 14.]
n_siblings_spouses  : [0 0 0 0 1]
class               : [b'Third' b'Second' b'Second' b'Third' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
alone               : [b'y' b'y' b'y' b'y' b'n']


In [10]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [38. 33. 31. 60.  3.]
n_siblings_spouses  : [0. 0. 1. 1. 4.]
parch               : [0. 0. 0. 1. 2.]
fare                : [ 7.05   9.5   57.    79.2   31.388]


In [11]:
example_batch, labels_batch = next(iter(temp_dataset)) 

In [12]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

In [13]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

[[16.     1.     3.    34.375]
 [50.     0.     0.    28.712]
 [24.     0.     0.     8.05 ]
 [29.     1.     0.     7.046]
 [22.     0.     1.    55.   ]]

[0 0 0 0 1]


In [14]:
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'male' b'female' b'male']
age                 : [34. 28. 26. 28. 36.]
n_siblings_spouses  : [1 0 0 0 0]
parch               : [1 0 0 0 1]
fare                : [ 14.4    13.      8.05    7.75  512.329]
class               : [b'Third' b'Second' b'Third' b'Third' b'First']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'B']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Queenstown' b'Cherbourg']
alone               : [b'n' b'y' b'y' b'y' b'n']


In [15]:
example_batch, labels_batch = next(iter(temp_dataset)) 

In [16]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [17]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [18]:
show_batch(packed_train_data)

sex                 : [b'male' b'male' b'male' b'male' b'male']
class               : [b'Second' b'Third' b'First' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Cherbourg' b'Cherbourg' b'Cherbourg']
alone               : [b'y' b'y' b'y' b'n' b'n']
numeric             : [[39.     0.     0.    13.   ]
 [22.     0.     0.     9.   ]
 [71.     0.     0.    49.504]
 [28.     1.     1.    15.246]
 [28.     2.     0.    21.679]]


In [19]:
example_batch, labels_batch = next(iter(packed_train_data)) 

In [20]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [21]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [22]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std


In [23]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7f004de3aef0>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [24]:
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[ 28.   ,   0.   ,   0.   , 227.525],
       [ 25.   ,   1.   ,   0.   ,  26.   ],
       [ 19.   ,   0.   ,   0.   ,   7.775],
       [ 33.   ,   0.   ,   0.   ,  12.275],
       [ 19.   ,   0.   ,   0.   ,   8.158]], dtype=float32)>

In [25]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-0.13 , -0.474, -0.479,  3.538],
       [-0.37 ,  0.395, -0.479, -0.154],
       [-0.85 , -0.474, -0.479, -0.487],
       [ 0.269, -0.474, -0.479, -0.405],
       [-0.85 , -0.474, -0.479, -0.48 ]], dtype=float32)

In [26]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}


In [27]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [28]:
# See what you just created.
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [29]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0.]


In [30]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [31]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 1.     0.     1.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     1.     0.     0.    -0.13  -0.474
 -0.479  3.538  1.     0.   ]


In [32]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1),
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

In [33]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [34]:
model.fit(train_data, epochs=20)

Epoch 1/20
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
126/126 [==============================] - 2s 5ms/step - loss: 0.5089 - accuracy: 0.7193
Epoch 2/20
126/126 [==============================] - 0s 2ms/step - loss: 0.4164 - accuracy: 0.8293
Epoch 3/20
126/126 [==============================] - 0s 2ms/step - loss: 0.4024 - accuracy: 0.8198
Epoch 4/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3938 - accuracy: 0.8293
Epoch 5/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3754 - accuracy: 0.8485
Epoch 6/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3723 - accuracy: 0.8453
Epoch 7/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3646 - accuracy: 0.8469
Epoch 8/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3628 - accuracy: 0.8469
Epoch 9/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3522 - accurac

In [35]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

Consider rewriting this model with the Functional API.
53/53 [==============================] - 0s 2ms/step - loss: 0.4462 - accuracy: 0.8485


Test Loss 0.44620466232299805, Test Accuracy 0.8484848737716675


In [36]:
predictions = model.predict(test_data)

# Show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  prediction = tf.sigmoid(prediction).numpy()
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))


Consider rewriting this model with the Functional API.
Predicted survival: 12.87%  | Actual outcome:  DIED
Predicted survival: 99.74%  | Actual outcome:  DIED
Predicted survival: 99.11%  | Actual outcome:  DIED
Predicted survival: 99.96%  | Actual outcome:  DIED
Predicted survival: 90.36%  | Actual outcome:  SURVIVED
